In [4]:
import sys
#!{sys.executable} -m pip install sagemaker

In [5]:
import sagemaker
import os
from dataclasses import dataclass, asdict
import argparse
from cohorts import *
from sagemaker.workflow.pipeline_context import LocalPipelineSession
import sagemaker
import os
import subprocess
from sagemaker.local import LocalSession
from sagemaker.estimator import Estimator
import boto3


In [6]:
import numpy
from sagemaker.mxnet import MXNetModel

model_location = 's3://mybucket/my_model.tar.gz'
code_location = 's3://mybucket/sourcedir.tar.gz'
s3_model = MXNetModel(model_data=model_location, role='SageMakerRole',
                      entry_point='mnist.py', source_dir=code_location)

predictor = s3_model.deploy(initial_instance_count=1, instance_type='local')
data = numpy.zeros(shape=(1, 1, 28, 28))
predictor.predict(data)

# Tear down the endpoint container and delete the corresponding endpoint configuration
predictor.delete_endpoint()

# Deletes the model
predictor.delete_model()

ValueError: framework_version or py_version was None, yet image_uri was also None. Either specify both framework_version and py_version, or specify image_uri.

In [2]:
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'
instance_type = "local"

try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local"

        sagemaker_session = LocalSession()
        sagemaker_session.config = {'local': {'local_code': True}}
except:
    pass

print("Instance type = " + instance_type)

Windows Support for Local Mode is Experimental


Instance type = local


## Build container entrypoint

In [3]:
dataset = Panda()
options = asdict(dataset)
container_entry_point=["python3", "./preprocessing/infer_entries.py"]

# Convert dictionary to list format
for key, value in options.items():
    if value:  # Only add the key for True values
        container_entry_point.extend([f"--{key}", str(value)])
print(container_entry_point)

['python3', './preprocessing/infer_entries.py', '--tarball_path', '/opt/ml/input/data/tarballs/panda.tar', '--output_root', '/opt/ml/output/preprocessing/', '--remove', '/opt/ml/input/data/restrict_files/downstream_panda_test.txt', '--name', 'panda']


## Build container

In [ ]:
preprocessing_image = "668452981977.dkr.ecr.us-east-1.amazonaws.com/rumc-stoic-p/sagemaker-daan-geijs:lightstream12"
sm_role = "AmazonSageMaker-ExecutionRole-20221113T171260"

sm_role = 'SageMakerRole'
instance_type = 'local'
docker_image = 'image_tag'
estimator = Estimator(
    image_uri=docker_image,
    container_entry_point=container_entry_point,
    role=sm_role,
    instance_count=1,
    volume_size = 1024,
    environment={
        'VIPS_CONCURRENCY': '20',
        'OMP_NUM_THREADS': '4'
    },
    #metrics_definitions = metrics, 
    instance_type=instance_type,
    # use_spot_instances=True,
    # max_wait = 86400,
    output_path = "s3://rumc-stoic-p-sagemaker-daan-geijs/output/models",
    checkpoint_s3_uri = "s3://rumc-stoic-p-sagemaker-daan-geijs/output/checkpoints",
)

In [ ]:
s3_root = "s3://rumc-stoic-p-sagemaker-daan-geijs/input/vlfm"

data_channels = {
    'tarballs': s3_root+'/data/tarballs/',
    'filter_files': s3_root+'/data/filter_files/',
    'cohorts': s3_root+'/data/cohorts',
}
print(data_channels)
estimator.fit(s3_root)

In [8]:
# s3_root = "s3://tiger-training/"
# s3_root

# estimator.fit(s3_root)
# job_name = estimator.latest_training_job.name
# print("Training job name: {}".format(job_name))

# client = estimator.sagemaker_session.sagemaker_client

# description = client.describe_training_job(TrainingJobName=job_name)



's3://tiger-training/'